### Campo elegido para el proyecto: Agricultura

Asistente para analizar artículos de agrícultura

In [2]:
"""
Sistema Agentic AI Multi-Agente para Procesamiento de Documentos
Universidad Nacional de Colombia - PLN
Integración con Groq (modelos Llama) y embeddings locales
"""

import os
import json
from datetime import datetime
from typing import List, Dict, Any, Optional
from pathlib import Path
from dotenv import load_dotenv

# Sentence Transformers para embeddings locales
from sentence_transformers import SentenceTransformer
import numpy as np

# LangChain imports
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Import correcto para RecursiveCharacterTextSplitter
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    try:
        from langchain.text_splitter import RecursiveCharacterTextSplitter
    except ImportError:
        from langchain_core.text_splitter import RecursiveCharacterTextSplitter

# Importaciones para cargar documentos
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredHTMLLoader

# Rich para visualización
from rich.console import Console
from rich.panel import Panel
from rich.tree import Tree

# Cargar variables de entorno
load_dotenv()

# Configuración de consola
console = Console()


class LocalEmbeddings:
    """
    Clase para generar embeddings locales usando Sentence Transformers
    Compatible con FAISS y LangChain
    No requiere API keys y funciona offline
    """
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Inicializar modelo de embeddings
        Modelos recomendados:
        - all-MiniLM-L6-v2: Rápido, ligero (80MB), buena calidad
        - paraphrase-multilingual-MiniLM-L12-v2: Mejor para español
        """
        console.print(f"[yellow]Cargando modelo de embeddings: {model_name}...[/yellow]")
        self.model = SentenceTransformer(model_name)
        console.print("[green]✓ Modelo de embeddings cargado[/green]")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generar embeddings para una lista de documentos"""
        embeddings = self.model.encode(
            texts, 
            show_progress_bar=True, 
            convert_to_numpy=True,
            batch_size=32
        )
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Generar embedding para una consulta"""
        embedding = self.model.encode([text], convert_to_numpy=True)
        return embedding[0].tolist()
    
    def __call__(self, text: str) -> List[float]:
        """Hacer la clase callable para compatibilidad con FAISS"""
        return self.embed_query(text)


class TraceabilityLogger:
    """Clase para mantener trazabilidad completa del sistema"""
    
    def __init__(self):
        self.trace_log = []
        self.start_time = None
        
    def start_trace(self, query: str):
        """Iniciar trazabilidad de una consulta"""
        self.start_time = datetime.now()
        self.trace_log = [{
            "timestamp": self.start_time.isoformat(),
            "event": "query_received",
            "query": query
        }]
        
    def log_event(self, agent: str, action: str, details: Dict[str, Any]):
        """Registrar evento en el log de trazabilidad"""
        self.trace_log.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "action": action,
            "details": details
        })
        
    def get_trace(self) -> List[Dict]:
        """Obtener el log completo de trazabilidad"""
        return self.trace_log
    
    def print_trace(self):
        """Imprimir trazabilidad de forma visual"""
        tree = Tree("🔍 [bold blue]Trazabilidad del Sistema[/bold blue]")
        
        for entry in self.trace_log:
            if entry.get("event") == "query_received":
                tree.add(f"[green]Query recibida:[/green] {entry['query']}")
            else:
                agent_node = tree.add(f"[yellow]{entry['agent']}[/yellow]")
                agent_node.add(f"[cyan]Acción:[/cyan] {entry['action']}")
                if entry.get("details"):
                    details_str = json.dumps(entry["details"], indent=2, ensure_ascii=False)
                    agent_node.add(f"[magenta]Detalles:[/magenta]\n{details_str}")
        
        console.print(tree)


class DocumentIndexer:
    """
    Agente 1: Consumo e Indexación de Documentos
    - Carga documentos (PDF/TXT/HTML)
    - Limpia, segmenta en chunks y genera embeddings
    - Indexa en FAISS
    """
    
    def __init__(self, embeddings_model, trace_logger: TraceabilityLogger):
        self.embeddings = embeddings_model
        self.trace_logger = trace_logger
        self.vector_store = None
        self.documents = []
        
    def load_documents(self, directory_path: str) -> List[Document]:
        """Cargar documentos desde un directorio"""
        console.print(f"[bold green]📂 Cargando documentos desde: {directory_path}[/bold green]")
        
        documents = []
        path = Path(directory_path)
        
        # Extensiones soportadas
        loaders_map = {
            '.pdf': PyPDFLoader,
            '.txt': TextLoader,
            '.html': UnstructuredHTMLLoader,
            '.htm': UnstructuredHTMLLoader
        }
        
        file_count = 0
        for file_path in path.rglob('*'):
            if file_path.is_file() and file_path.suffix.lower() in loaders_map:
                try:
                    loader_class = loaders_map[file_path.suffix.lower()]
                    
                    if file_path.suffix.lower() == '.txt':
                        loader = loader_class(str(file_path), encoding='utf-8')
                    else:
                        loader = loader_class(str(file_path))
                    
                    docs = loader.load()
                    documents.extend(docs)
                    file_count += 1
                    
                    if file_count % 10 == 0:
                        console.print(f"  ✓ Cargados {file_count} documentos...")
                        
                except Exception as e:
                    console.print(f"[red]  ✗ Error al cargar {file_path.name}: {str(e)}[/red]")
                    continue
        
        console.print(f"[bold green]✓ Total de documentos cargados: {file_count}[/bold green]")
        
        self.trace_logger.log_event(
            agent="DocumentIndexer",
            action="load_documents",
            details={"total_documents": file_count, "directory": directory_path}
        )
        
        self.documents = documents
        return documents
    
    def process_and_index(self, documents: List[Document], chunk_size: int = 1000, chunk_overlap: int = 200):
        """Procesar documentos: chunking + embeddings + indexación en FAISS"""
        console.print("[bold yellow]🔨 Procesando documentos...[/bold yellow]")
        
        # Dividir en chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        
        chunks = text_splitter.split_documents(documents)
        console.print(f"  ✓ Documentos divididos en {len(chunks)} chunks")
        
        # Crear vector store con FAISS
        console.print("[bold yellow]🧮 Generando embeddings e indexando en FAISS...[/bold yellow]")
        self.vector_store = FAISS.from_documents(chunks, self.embeddings)
        console.print("[bold green]✓ Indexación completada[/bold green]")
        
        self.trace_logger.log_event(
            agent="DocumentIndexer",
            action="process_and_index",
            details={
                "total_chunks": len(chunks),
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap
            }
        )
        
        return self.vector_store
    
    def save_index(self, path: str = "faiss_index"):
        """Guardar índice FAISS en disco"""
        if self.vector_store:
            self.vector_store.save_local(path)
            console.print(f"[green]✓ Índice guardado en: {path}[/green]")
    
    def load_index(self, path: str = "faiss_index"):
        """Cargar índice FAISS desde disco"""
        try:
            self.vector_store = FAISS.load_local(
                path, 
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            console.print(f"[green]✓ Índice cargado desde: {path}[/green]")
            return self.vector_store
        except Exception as e:
            console.print(f"[red]✗ Error al cargar índice: {str(e)}[/red]")
            return None


class QueryClassifier:
    """
    Agente 2: Clasificador de Consultas
    Clasifica la intención del usuario en 4 categorías:
    - búsqueda: Consulta específica en documentos
    - resumen: Resumen de documentos
    - comparación: Comparar documentos
    - general: Pregunta general sin necesidad de RAG
    
    Usa Groq Llama 3.3 70B para interpretación profunda del lenguaje
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para interpretación profunda del lenguaje
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def classify(self, query: str) -> Dict[str, Any]:
        """Clasificar la intención de la consulta del usuario"""
        
        classification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un clasificador de consultas experto. Debes clasificar la intención del usuario en una de estas categorías:

1. "busqueda": El usuario solicita información específica, hechos o datos contenidos en documentos
2. "resumen": El usuario quiere un resumen de uno o varios documentos
3. "comparacion": El usuario quiere comparar o contrastar información de diferentes documentos
4. "general": El usuario hace una pregunta general que no requiere buscar en documentos específicos

Ejemplos:
- "¿Cuál es la capital de Francia?" -> general
- "¿Qué dice el documento sobre las causas de la guerra?" -> busqueda
- "Resume el contenido del artículo sobre IA" -> resumen
- "Compara las conclusiones de estos dos estudios" -> comparacion
- "Explícame qué es la fotosíntesis" -> general
- "¿Qué información hay sobre cambio climático?" -> busqueda

Responde SOLO con un JSON en este formato exacto:
{{"intencion": "busqueda|resumen|comparacion|general", "confianza": 0.95, "razonamiento": "breve explicación"}}"""),
            ("user", "{query}")
        ])
        
        chain = classification_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"query": query})
        
        # Parsear respuesta
        try:
            # Limpiar respuesta si tiene markdown
            response_clean = response.strip()
            if response_clean.startswith("```json"):
                response_clean = response_clean[7:]
            if response_clean.endswith("```"):
                response_clean = response_clean[:-3]
            
            result = json.loads(response_clean.strip())
            
            self.trace_logger.log_event(
                agent="QueryClassifier",
                action="classify_intent",
                details={
                    "query": query,
                    "clasificacion": result
                }
            )
            
            console.print(Panel(
                f"[bold cyan]Intención:[/bold cyan] {result['intencion']}\n"
                f"[bold cyan]Confianza:[/bold cyan] {result['confianza']}\n"
                f"[bold cyan]Razonamiento:[/bold cyan] {result['razonamiento']}",
                title="🎯 Clasificación de Query"
            ))
            
            return result
            
        except json.JSONDecodeError:
            console.print("[red]Error al parsear clasificación, usando 'busqueda' por defecto[/red]")
            return {
                "intencion": "busqueda",
                "confianza": 0.5,
                "razonamiento": "Error en clasificación, asumiendo búsqueda"
            }


class SemanticRetriever:
    """
    Agente 3: Recuperador Semántico
    - Ejecuta búsqueda de similaridad semántica en FAISS
    - Selecciona documentos más relevantes
    
    Usa Groq Llama 3.1 8B para optimizar velocidad de recuperación
    """
    
    def __init__(self, vector_store: FAISS, trace_logger: TraceabilityLogger):
        self.vector_store = vector_store
        self.trace_logger = trace_logger
        # Groq Llama 3.1 8B para velocidad de recuperación
        self.llm = ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def retrieve(self, query: str, k: int = 5) -> List[Document]:
        """Recuperar documentos relevantes mediante búsqueda semántica"""
        
        console.print(f"[bold yellow]🔎 Buscando documentos relevantes (top-{k})...[/bold yellow]")
        
        # Búsqueda de similaridad en FAISS
        relevant_docs = self.vector_store.similarity_search(query, k=k)
        
        self.trace_logger.log_event(
            agent="SemanticRetriever",
            action="retrieve_documents",
            details={
                "query": query,
                "num_docs_retrieved": len(relevant_docs),
                "doc_sources": [doc.metadata.get('source', 'unknown') for doc in relevant_docs]
            }
        )
        
        console.print(f"[green]✓ Recuperados {len(relevant_docs)} documentos relevantes[/green]")
        
        return relevant_docs


class RAGResponseGenerator:
    """
    Agente 4: Generador de Respuestas con RAG
    - Construye respuesta combinando query + contexto recuperado
    - Produce respuestas justificadas con citas
    
    Usa Groq Llama 3.3 70B para generar respuestas rápidas basadas en contexto
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para respuestas de alta calidad
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.3,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def generate_response(self, query: str, context_docs: List[Document], intent: str) -> str:
        """Generar respuesta usando RAG"""
        
        console.print("[bold yellow]✍️ Generando respuesta con RAG...[/bold yellow]")
        
        # Preparar contexto
        context = "\n\n".join([
            f"--- Documento {i+1} (Fuente: {doc.metadata.get('source', 'desconocida')}) ---\n{doc.page_content}"
            for i, doc in enumerate(context_docs)
        ])
        
        # Prompt según intención
        if intent == "resumen":
            system_prompt = """Eres un asistente experto en resumir documentos. 
            Genera un resumen claro y conciso del contenido proporcionado.
            Incluye las ideas principales y cita las fuentes cuando sea apropiado."""
        elif intent == "comparacion":
            system_prompt = """Eres un asistente experto en análisis comparativo.
            Compara y contrasta la información de los documentos proporcionados.
            Identifica similitudes, diferencias y cita las fuentes de cada afirmación."""
        else:  # búsqueda
            system_prompt = """Eres un asistente experto en responder preguntas basándote en documentos.
            Responde la pregunta del usuario usando ÚNICAMENTE la información del contexto proporcionado.
            IMPORTANTE: Cita las fuentes de donde obtienes la información.
            Si la información no está en el contexto, indícalo claramente."""
        
        rag_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("user", """Contexto de los documentos:
{context}

Pregunta del usuario: {query}

Responde de forma clara, precisa y justificada con citas a las fuentes.""")
        ])
        
        chain = rag_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"context": context, "query": query})
        
        self.trace_logger.log_event(
            agent="RAGResponseGenerator",
            action="generate_response",
            details={
                "query": query,
                "intent": intent,
                "num_context_docs": len(context_docs),
                "response_length": len(response)
            }
        )
        
        console.print("[green]✓ Respuesta generada[/green]")
        
        return response


class ResponseVerifier:
    """
    Agente 5: Verificador/Crítico de Respuestas
    - Evalúa si la respuesta está respaldada por el contexto
    - Verifica coherencia y evita alucinaciones
    - Valida cumplimiento de requerimientos
    
    Usa Groq Llama 3.3 70B para razonamiento y validación compleja
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para razonamiento y validación compleja
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def verify(self, query: str, response: str, context_docs: List[Document]) -> Dict[str, Any]:
        """Verificar calidad y coherencia de la respuesta"""
        
        console.print("[bold yellow]✅ Verificando respuesta...[/bold yellow]")
        
        context = "\n\n".join([doc.page_content for doc in context_docs])
        
        verification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un evaluador crítico de respuestas. Tu trabajo es verificar:

1. ¿La respuesta está respaldada por el contexto proporcionado?
2. ¿La respuesta es coherente y lógica?
3. ¿Hay signos de alucinación (información inventada no presente en el contexto)?
4. ¿La respuesta responde adecuadamente a la pregunta del usuario?

Responde con un JSON en este formato:
{{
    "es_valida": true/false,
    "confianza": 0.0-1.0,
    "problemas_detectados": ["lista de problemas si existen"],
    "sugerencias_mejora": ["sugerencias si es_valida es false"],
    "razonamiento": "explicación de tu evaluación"
}}"""),
            ("user", """Contexto:
{context}

Pregunta del usuario: {query}

Respuesta a evaluar: {response}

Evalúa la respuesta.""")
        ])
        
        chain = verification_prompt | self.llm | StrOutputParser()
        verification_result = chain.invoke({
            "context": context,
            "query": query,
            "response": response
        })
        
        try:
            # Limpiar respuesta
            verification_clean = verification_result.strip()
            if verification_clean.startswith("```json"):
                verification_clean = verification_clean[7:]
            if verification_clean.endswith("```"):
                verification_clean = verification_clean[:-3]
            
            result = json.loads(verification_clean.strip())
            
            self.trace_logger.log_event(
                agent="ResponseVerifier",
                action="verify_response",
                details=result
            )
            
            if result["es_valida"]:
                console.print("[bold green]✓ Respuesta verificada exitosamente[/bold green]")
            else:
                console.print("[bold red]✗ Respuesta requiere mejoras[/bold red]")
                console.print(f"[yellow]Problemas: {', '.join(result['problemas_detectados'])}[/yellow]")
            
            return result
            
        except json.JSONDecodeError:
            console.print("[red]Error al parsear verificación[/red]")
            return {
                "es_valida": True,
                "confianza": 0.5,
                "problemas_detectados": [],
                "sugerencias_mejora": [],
                "razonamiento": "Error en verificación, aceptando respuesta"
            }


class Orchestrator:
    """
    Agente Orquestador Principal
    - Administra el flujo completo del sistema
    - Determina qué agente ejecutar según la consulta
    - Coordina la interacción entre agentes
    """
    
    def __init__(
        self,
        indexer: DocumentIndexer,
        classifier: QueryClassifier,
        retriever: SemanticRetriever,
        generator: RAGResponseGenerator,
        verifier: ResponseVerifier,
        trace_logger: TraceabilityLogger
    ):
        self.indexer = indexer
        self.classifier = classifier
        self.retriever = retriever
        self.generator = generator
        self.verifier = verifier
        self.trace_logger = trace_logger
        
        # LLM para decisiones del orquestador (Llama 3.1 8B por velocidad)
        self.llm = ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def process_query(self, query: str, max_retries: int = 2) -> Dict[str, Any]:
        """Procesar consulta del usuario orquestando todos los agentes"""
        
        console.print(Panel(
            f"[bold white]{query}[/bold white]",
            title="📝 Query del Usuario",
            border_style="blue"
        ))
        
        # Iniciar trazabilidad
        self.trace_logger.start_trace(query)
        
        # Paso 1: Clasificar intención
        classification = self.classifier.classify(query)
        intent = classification["intencion"]
        
        self.trace_logger.log_event(
            agent="Orchestrator",
            action="route_query",
            details={"intent": intent}
        )
        
        # Paso 2: Rutear según intención
        if intent == "general":
            # Respuesta directa sin RAG
            return self._handle_general_query(query)
        else:
            # Requiere RAG: búsqueda, resumen o comparación
            return self._handle_rag_query(query, intent, max_retries)
    
    def _handle_general_query(self, query: str) -> Dict[str, Any]:
        """Manejar consulta general sin necesidad de RAG"""
        
        console.print("[bold cyan]💬 Procesando consulta general (sin RAG)...[/bold cyan]")
        
        # Usar Groq Llama 3.3 70B para respuestas generales de calidad
        llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.7,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
        general_prompt = ChatPromptTemplate.from_messages([
            ("system", "Eres un asistente útil y conocedor. Responde las preguntas del usuario de manera clara y precisa."),
            ("user", "{query}")
        ])
        
        chain = general_prompt | llm | StrOutputParser()
        response = chain.invoke({"query": query})
        
        self.trace_logger.log_event(
            agent="Orchestrator",
            action="handle_general_query",
            details={"response_length": len(response)}
        )
        
        return {
            "query": query,
            "intent": "general",
            "response": response,
            "documents_used": [],
            "verification": {"es_valida": True, "confianza": 1.0},
            "trace": self.trace_logger.get_trace()
        }
    
    def _handle_rag_query(self, query: str, intent: str, max_retries: int) -> Dict[str, Any]:
        """Manejar consulta que requiere RAG"""
        
        console.print(f"[bold cyan]🔄 Procesando consulta con RAG (intención: {intent})...[/bold cyan]")
        
        # Paso 1: Recuperar documentos relevantes
        k = 5 if intent == "busqueda" else 10  # Más docs para resumen/comparación
        relevant_docs = self.retriever.retrieve(query, k=k)
        
        if not relevant_docs:
            return {
                "query": query,
                "intent": intent,
                "response": "No se encontraron documentos relevantes para responder a tu consulta.",
                "documents_used": [],
                "verification": {"es_valida": False, "confianza": 0.0},
                "trace": self.trace_logger.get_trace()
            }
        
        # Paso 2: Generar respuesta con RAG (con loop de verificación)
        for attempt in range(max_retries + 1):
            console.print(f"[cyan]Intento {attempt + 1}/{max_retries + 1} de generación...[/cyan]")
            
            response = self.generator.generate_response(query, relevant_docs, intent)
            
            # Paso 3: Verificar respuesta
            verification = self.verifier.verify(query, response, relevant_docs)
            
            if verification["es_valida"] or attempt == max_retries:
                # Respuesta válida o se alcanzó máximo de intentos
                return {
                    "query": query,
                    "intent": intent,
                    "response": response,
                    "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs],
                    "verification": verification,
                    "attempts": attempt + 1,
                    "trace": self.trace_logger.get_trace()
                }
            
            console.print("[yellow]⚠️ Respuesta no válida, regenerando...[/yellow]")
        
        # Esto no debería alcanzarse nunca
        return {
            "query": query,
            "intent": intent,
            "response": response,
            "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs],
            "verification": verification,
            "trace": self.trace_logger.get_trace()
        }


def main():
    """Función principal del sistema"""
    
    console.print(Panel(
        "[bold cyan]Sistema Agentic AI Multi-Agente[/bold cyan]\n"
        "[white]Universidad Nacional de Colombia - PLN[/white]\n"
        "[yellow]Procesamiento y Análisis Inteligente de Documentos[/yellow]",
        title="🤖 Bienvenido",
        border_style="green"
    ))
    
    # Verificar variable de entorno
    if not os.getenv("GROQ_API_KEY"):
        console.print("[bold red]Error: GROQ_API_KEY no configurada[/bold red]")
        console.print("[yellow]Por favor, añade tu API key de Groq al archivo .env[/yellow]")
        console.print("[yellow]Obténla en: https://console.groq.com/keys[/yellow]")
        return
    
    # Inicializar componentes
    console.print("\n[bold cyan]Inicializando sistema...[/bold cyan]")
    
    # Embeddings locales (no requiere API)
    embeddings = LocalEmbeddings(model_name="all-MiniLM-L6-v2")
    
    trace_logger = TraceabilityLogger()
    
    # Inicializar agentes
    indexer = DocumentIndexer(embeddings, trace_logger)
    
    # Obtener rutas desde .env o usar defaults relativos
    documents_path = os.getenv("DOCUMENTS_PATH", "./data")
    index_path = os.getenv("FAISS_INDEX_PATH", "./faiss_index")
    
    # Convertir a ruta absoluta para mejor manejo
    documents_path = os.path.abspath(documents_path)
    index_path = os.path.abspath(index_path)
    
    console.print(f"[cyan]Ruta de documentos: {documents_path}[/cyan]")
    console.print(f"[cyan]Ruta de índice: {index_path}[/cyan]")
    
    # Cargar o crear índice
    vector_store = indexer.load_index(index_path)
    
    if vector_store is None:
        # Verificar que existe el directorio de documentos
        if not os.path.exists(documents_path):
            console.print(f"[bold red]Error: No se encuentra el directorio de documentos:[/bold red]")
            console.print(f"[yellow]{documents_path}[/yellow]")
            console.print(f"[cyan]Crea la carpeta 'data' en el directorio del proyecto y coloca tus documentos allí[/cyan]")
            return
        
        # Cargar y procesar documentos
        documents = indexer.load_documents(documents_path)
        
        if not documents:
            console.print("[bold red]No se encontraron documentos para procesar[/bold red]")
            console.print("[yellow]Formatos soportados: PDF, TXT, HTML[/yellow]")
            return
        
        vector_store = indexer.process_and_index(documents)
        indexer.save_index(index_path)
    
    # Inicializar resto de agentes
    classifier = QueryClassifier(trace_logger)
    retriever = SemanticRetriever(vector_store, trace_logger)
    generator = RAGResponseGenerator(trace_logger)
    verifier = ResponseVerifier(trace_logger)
    
    # Inicializar orquestador
    orchestrator = Orchestrator(
        indexer, classifier, retriever, generator, verifier, trace_logger
    )
    
    console.print("[bold green]✓ Sistema inicializado exitosamente[/bold green]\n")
    
    # Loop interactivo
    while True:
        console.print("\n" + "="*80)
        query = console.input("[bold cyan]👤 Ingresa tu consulta (o 'salir' para terminar): [/bold cyan]")
        
        if query.lower() in ['salir', 'exit', 'quit']:
            console.print("[bold green]¡Hasta luego![/bold green]")
            break
        
        if not query.strip():
            continue
        
        # Procesar consulta
        result = orchestrator.process_query(query)
        
        # Mostrar respuesta
        console.print("\n")
        console.print(Panel(
            f"[bold white]{result['response']}[/bold white]",
            title="🤖 Respuesta del Sistema",
            border_style="green"
        ))
        
        # Mostrar información adicional
        if result.get("documents_used"):
            console.print(f"\n[cyan]📚 Documentos utilizados: {len(result['documents_used'])}[/cyan]")
        
        console.print(f"[cyan]🎯 Intención detectada: {result['intent']}[/cyan]")
        
        if result.get("verification"):
            ver = result["verification"]
            console.print(f"[cyan]✅ Confianza de verificación: {ver.get('confianza', 0):.2f}[/cyan]")
        
        # Mostrar trazabilidad
        if console.input("\n[yellow]¿Mostrar trazabilidad completa? (s/n): [/yellow]").lower() == 's':
            trace_logger.print_trace()


if __name__ == "__main__":
    main()

╭───────────────────────────────────────────────── 🤖 Bienvenido ─────────────────────────────────────────────────╮
│ Sistema Agentic AI Multi-Agente                                                                                 │
│ Universidad Nacional de Colombia - PLN                                                                          │
│ Procesamiento y Análisis Inteligente de Documentos                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Inicializando sistema...

Cargando modelo de embeddings: all-MiniLM-L6-v2...

✓ Modelo de embeddings cargado

Ruta de documentos: c:\Users\juana\OneDrive\Documentos\UNAL - 
GITHUB\UNAL_2025_2\PLN\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\data

...

¿Mostrar trazabilidad completa? (s/n): 

🔍 Trazabilidad del Sistema
├── Query recibida: Dame un pequeño resumen
├── QueryClassifier
│   ├── Acción: classify_intent
│   └── Detalles:
│       {
│         "query": "Dame un pequeño resumen",
│         "clasificacion": {
│           "intencion": "resumen",
│           "confianza": 0.8,
│           "razonamiento": "El usuario solicita un resumen sin especificar un documento en particular, pero la 
│       palabra clave 'resumen' indica claramente su intención"
│         }
│       }
├── Orchestrator
│   ├── Acción: route_query
│   └── Detalles:
│       {
│         "intent": "resumen"
│       }
├── SemanticRetriever
│   ├── Acción: retrieve_documents
│   └── Detalles:
│       {
│         "query": "Dame un pequeño resumen",
│         "num_docs_retrieved": 10,
│         "doc_sources": [
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-CuidadoTrabajoYGeneroInsercionYCentralidadLaboralE-9093911.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-CuidadoTrabajoYGeneroInsercionYCentralidadLaboralE-9093911.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-PerfilDeLasInvestigacionesEnProduccionSostenibleCo-8906664.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-SobreElTratamientoDeLasCitasEvacuadasDeLasEdicione-10243894 (1).pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-SobreElTratamientoDeLasCitasEvacuadasDeLasEdicione-10243894.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-CuidadoTrabajoYGeneroInsercionYCentralidadLaboralE-9093911.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\agricu
│       ltura y protoindustrializacion.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-AgriculturaYProtoindustrializacion-5319028.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-CuidadoTrabajoYGeneroInsercionYCentralidadLaboralE-9093911.pdf",
│           "c:\\Users\\juana\\OneDrive\\Documentos\\UNAL - 
│       GITHUB\\UNAL_2025_2\\PLN\\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\\data\\Dialne
│       t-GerardoBudowski-9749513.pdf"
│         ]
│       }
├── RAGResponseGenerator
│   ├── Acción: generate_response
│   └── Detalles:
│       {
│         "query": "Dame un pequeño resumen",
│         "intent": "resumen",
│         "num_context_docs": 10,
│         "response_length": 1578
│       }
└── ResponseVerifier
    ├── Acción: verify_response
    └── Detalles:
        {
          "es_valida": true,
          "confianza": 0.9,
          "problemas_detectados": [],
          "sugerencias_mejora": [],
          "razonamiento": "La respuesta proporciona un resumen claro y conciso de los temas relacionados con la 
        igualdad de género, el trabajo y la educación. Se basa en información proporcionada en el contexto y 
        utiliza referencias específicas para respaldar sus 

================================================================================

👤 Ingresa tu consulta (o 'salir' para terminar): 

¡Hasta luego!

In [3]:
import json
import os
from datetime import datetime
from rich.console import Console
from rich.table import Table
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# NOTA: Ya no necesitamos importar las clases (DocumentIndexer, etc.) 
# porque al estar en el mismo notebook, ya están en memoria si ejecutaste las celdas anteriores.

console = Console()

class TestCaseRunner:
    """Ejecutor de casos de prueba con documentación automática"""
    
    def __init__(self, orchestrator):
        self.orchestrator = orchestrator
        self.test_results = []
        
    def run_test_case(self, test_id: int, query: str, expected_intent: str, description: str):
        """Ejecutar un caso de prueba y documentar resultado"""
        
        console.print(f"\n{'='*80}")
        console.print(f"[bold cyan]CASO DE PRUEBA #{test_id}[/bold cyan]")
        console.print(f"[yellow]Descripción:[/yellow] {description}")
        console.print(f"[yellow]Query:[/yellow] {query}")
        console.print(f"[yellow]Intención esperada:[/yellow] {expected_intent}")
        console.print(f"{'='*80}\n")
        
        start_time = datetime.now()
        
        try:
            # Ejecutamos el orquestador
            result = self.orchestrator.process_query(query)
            end_time = datetime.now()
            duration = (end_time - start_time).total_seconds()
            
            # Validamos si la intención coincide
            success = result["intent"] == expected_intent
            
            test_result = {
                "test_id": test_id,
                "description": description,
                "query": query,
                "expected_intent": expected_intent,
                "actual_intent": result["intent"],
                "response": result.get("response", "Sin respuesta"),
                "documents_used": len(result.get("documents_used", [])),
                "verification": result.get("verification", {}),
                "duration_seconds": duration,
                "trace": result.get("trace", []),
                "success": success,
                "timestamp": start_time.isoformat()
            }
            
            self.test_results.append(test_result)
            
            console.print(f"\n[bold green]✓ Caso de prueba completado en {duration:.2f}s[/bold green]")
            console.print(f"[cyan]Intención detectada:[/cyan] {result['intent']} {'✓' if success else '✗'}")
            
            return test_result
            
        except Exception as e:
            console.print(f"[bold red]✗ Error en caso de prueba: {str(e)}[/bold red]")
            # Imprimir el traceback completo si es necesario para depurar
            import traceback
            traceback.print_exc()
            return None
    
    def generate_report(self, output_file: str = "test_results.json"):
        """Generar reporte de resultados"""
        
        # Guardar JSON
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.test_results, f, indent=2, ensure_ascii=False)
        
        console.print(f"\n[bold green]✓ Reporte guardado en: {output_file}[/bold green]")
        
        # Tabla resumen visual
        table = Table(title="Resumen de Casos de Prueba")
        table.add_column("ID", style="cyan")
        table.add_column("Descripción", style="white")
        table.add_column("Intención", style="yellow")
        table.add_column("Docs", style="magenta")
        table.add_column("Tiempo", style="green")
        table.add_column("Estado", style="bold")
        
        for result in self.test_results:
            status = "✓" if result["success"] else "✗"
            status_style = "green" if result["success"] else "red"
            
            desc_short = result["description"][:40] + "..." if len(result["description"]) > 40 else result["description"]
            
            table.add_row(
                str(result["test_id"]),
                desc_short,
                result["actual_intent"],
                str(result["documents_used"]),
                f"{result['duration_seconds']:.2f}s",
                f"[{status_style}]{status}[/{status_style}]"
            )
        
        console.print(table)
        
        # Estadísticas finales
        total = len(self.test_results)
        success_count = sum(1 for r in self.test_results if r["success"])
        avg_time = sum(r["duration_seconds"] for r in self.test_results) / total if total > 0 else 0
        
        console.print(f"\n[bold cyan]Estadísticas:[/bold cyan]")
        console.print(f"  Total de casos: {total}")
        console.print(f"  Exitosos: {success_count} ({success_count/total*100:.1f}%)")
        console.print(f"  Tiempo promedio: {avg_time:.2f}s")


# --- CONFIGURACIÓN Y EJECUCIÓN DE PRUEBAS ---

def run_test_suite():
    console.print("[bold cyan]Inicializando sistema para pruebas...[/bold cyan]")
    
    # 1. Configurar Embeddings (Asegúrate de que GOOGLE_API_KEY esté cargada)
    if not os.getenv("GOOGLE_API_KEY"):
        console.print("[red]ALERTA: No se encontró GOOGLE_API_KEY en variables de entorno[/red]")

    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    
    # 2. Instanciar Clases (Asume que las clases están definidas en celdas anteriores)
    trace_logger = TraceabilityLogger()
    indexer = DocumentIndexer(embeddings, trace_logger)
    
    # Intentar cargar el índice existente
    # Asegúrate de que la carpeta 'vectorstore/db_faiss' (o como la hayas llamado) exista
    try:
        vector_store = indexer.load_index("vectorstore/db_faiss") 
        if vector_store is None:
             # Fallback por si la ruta es diferente, ajusta según tu estructura
            vector_store = indexer.load_index("faiss_index")
            
        if vector_store is None:
            console.print("[bold red]ERROR CRÍTICO: No se pudo cargar el índice FAISS. Ejecuta la celda de indexación primero.[/bold red]")
            return
    except Exception as e:
        console.print(f"[red]Error cargando índice: {e}[/red]")
        return

    # 3. Inicializar Agentes
    classifier = QueryClassifier(trace_logger)
    retriever = SemanticRetriever(vector_store, trace_logger)
    generator = RAGResponseGenerator(trace_logger)
    verifier = ResponseVerifier(trace_logger)
    
    orchestrator = Orchestrator(
        indexer, classifier, retriever, generator, verifier, trace_logger
    )
    
    # 4. Preparar Runner
    runner = TestCaseRunner(orchestrator)
    
    # 5. Definir Casos de Prueba (Agricultura)
    test_cases = [
        {
            "query": "¿Qué es la agricultura sostenible?",
            "expected_intent": "general",
            "description": "Pregunta general sobre concepto básico"
        },
        {
            "query": "¿Qué técnicas de riego se mencionan en los documentos?",
            "expected_intent": "busqueda",
            "description": "Búsqueda específica de técnicas"
        },
        {
            "query": "Resume el contenido principal sobre control de plagas",
            "expected_intent": "resumen",
            "description": "Solicitud de resumen explícito"
        },
        {
            "query": "Compara los diferentes métodos de fertilización",
            "expected_intent": "comparacion",
            "description": "Comparación explícita"
        },
        {
            "query": "Explícame qué es la rotación de cultivos",
            "expected_intent": "general",
            "description": "Pregunta conceptual general"
        },
         {
            "query": "¿Cuáles son los principales cultivos descritos?",
            "expected_intent": "busqueda",
            "description": "Búsqueda de información fáctica"
        },
        {
            "query": "Contrasta la agricultura orgánica y la convencional",
            "expected_intent": "comparacion",
            "description": "Comparación de enfoques"
        },
        {
            "query": "¿Qué es la agroecología?",
            "expected_intent": "general",
            "description": "Pregunta de conocimiento general"
        },
        {
            "query": "¿Qué desafíos ambientales se mencionan?",
            "expected_intent": "busqueda",
            "description": "Búsqueda de hechos específicos"
        },
        {
            "query": "Resume las estrategias de conservación del agua",
            "expected_intent": "resumen",
            "description": "Resumen temático"
        }
    ]
    
    # 6. Ejecutar Loop
    for idx, test_case in enumerate(test_cases, 1):
        runner.run_test_case(
            test_id=idx,
            query=test_case["query"],
            expected_intent=test_case["expected_intent"],
            description=test_case["description"]
        )
    
    # 7. Reporte final
    runner.generate_report()
    console.print("\n[bold green]✓ Suite de pruebas completada exitosamente[/bold green]")

# Ejecutar la función
run_test_suite()

Inicializando sistema para pruebas...

✗ Error al cargar índice: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at 
D:\a\faiss-wheels\faiss-wheels\third-party\faiss\faiss\impl\io.cpp:70: Error: 'f' failed: could not open 
vectorstore\db_faiss\index.faiss for reading: No such file or directory

✓ Índice cargado desde: faiss_index

================================================================================

CASO DE PRUEBA #1

...

Estadísticas:

Total de casos: 3

Exitosos: 3 (100.0%)

Tiempo promedio: 3.63s

✓ Suite de pruebas completada exitosamente